In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('./data/survey_results_public.csv')
df_schema = pd.read_csv('./data/survey_results_schema.csv')

In [25]:
# filtering according to country and purchase influence columns

high_influence_df = df[(df['Country'] == 'Iran, Islamic Republic of...') & (df['PurchaseInfluence'] == 'I have a great deal of influence')]
high_influence_df

In [27]:
# filtering according to country and purchase influence columns

some_influence_df = df[(df['Country'] == 'Iran, Islamic Republic of...') & (df['PurchaseInfluence'] == 'I have some influence')]
some_influence_df

In [29]:
# table of employment distribution for developers with high influence

high_influence_employment = high_influence_df['Employment'].value_counts()
high_influence_employment

Employment
Employed, full-time                                                                             70
Independent contractor, freelancer, or self-employed                                            27
Employed, full-time;Independent contractor, freelancer, or self-employed                         7
Independent contractor, freelancer, or self-employed;Employed, part-time                         5
Employed, full-time;Employed, part-time                                                          2
Employed, part-time                                                                              2
I prefer not to say                                                                              2
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time     1
Name: count, dtype: int64

In [30]:
# table of employment distribution for developers with some influence

some_influence_employment = some_influence_df['Employment'].value_counts()
some_influence_employment

Employment
Employed, full-time                                                                             132
Independent contractor, freelancer, or self-employed                                             16
Employed, full-time;Independent contractor, freelancer, or self-employed                         13
Employed, part-time                                                                               5
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time      4
I prefer not to say                                                                               1
Employed, full-time;Employed, part-time                                                           1
Name: count, dtype: int64

In [31]:
# table of devtype distribution for developers with high influence

high_influence_devtype = high_influence_df['DevType'].value_counts()
high_influence_devtype

DevType
Developer, full-stack                            42
Developer, back-end                              28
Project manager                                   5
Developer, front-end                              4
Developer, desktop or enterprise applications     4
Engineering manager                               3
Product manager                                   3
Developer, mobile                                 2
Engineer, data                                    2
Research & Development role                       2
Security professional                             2
Data or business analyst                          2
Senior Executive (C-Suite, VP, etc.)              2
Data scientist or machine learning specialist     2
Developer, embedded applications or devices       2
Blockchain                                        1
Other (please specify):                           1
DevOps specialist                                 1
System administrator                              1
Engi

In [32]:
# table of devtype distribution for developers with some influence

some_influence_devtype = some_influence_df['DevType'].value_counts()
some_influence_devtype

DevType
Developer, back-end                              43
Developer, full-stack                            42
Developer, front-end                             23
Developer, mobile                                16
Developer, desktop or enterprise applications    14
Data scientist or machine learning specialist     7
Developer, embedded applications or devices       7
Developer, game or graphics                       2
Scientist                                         2
DevOps specialist                                 2
Other (please specify):                           2
Blockchain                                        1
Engineer, site reliability                        1
Engineering manager                               1
Data or business analyst                          1
Engineer, data                                    1
Project manager                                   1
Cloud infrastructure engineer                     1
Developer, QA or test                             1
Rese

In [33]:
# table of yearscodepro distribution for developers with high influence

high_influence_yearscodepro = high_influence_df['YearsCodePro'].value_counts()
high_influence_yearscodepro

YearsCodePro
8                   18
10                  12
5                   11
3                    9
6                    8
4                    8
7                    7
20                   6
2                    5
13                   4
Less than 1 year     4
9                    3
14                   3
1                    3
11                   2
16                   2
22                   2
12                   2
15                   1
17                   1
18                   1
24                   1
33                   1
29                   1
Name: count, dtype: int64

In [35]:
# table of yearscodepro distribution for developers with some influence

some_influence_yearscodepro = some_influence_df['YearsCodePro'].value_counts()
some_influence_yearscodepro

YearsCodePro
5                   32
3                   19
4                   17
6                   16
7                   15
8                   12
2                   12
10                  10
12                   8
11                   5
20                   5
15                   4
14                   3
1                    3
16                   2
Less than 1 year     2
21                   1
13                   1
25                   1
9                    1
22                   1
32                   1
18                   1
Name: count, dtype: int64

In [37]:
# table of edlevel distribution for developers with high influence

high_influence_edlevel = high_influence_df['EdLevel'].value_counts()
high_influence_edlevel

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          52
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       42
Some college/university study without earning a degree                                 7
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     5
Associate degree (A.A., A.S., etc.)                                                    4
Professional degree (JD, MD, Ph.D, Ed.D, etc.)                                         4
Something else                                                                         3
Name: count, dtype: int64

In [36]:
# table of edlevel distribution for developers with some influence

some_influence_edlevel = some_influence_df['EdLevel'].value_counts()
some_influence_edlevel

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          91
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       51
Some college/university study without earning a degree                                16
Associate degree (A.A., A.S., etc.)                                                    5
Professional degree (JD, MD, Ph.D, Ed.D, etc.)                                         3
Something else                                                                         2
Primary/elementary school                                                              2
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     2
Name: count, dtype: int64

In [38]:
# table of age distribution for developers with some influence

high_influence_age = high_influence_df['Age'].value_counts()
high_influence_age

Age
25-34 years old       60
35-44 years old       30
18-24 years old       18
45-54 years old        7
Under 18 years old     2
Name: count, dtype: int64

In [36]:
# table of yearscodepro distribution for edlevel with some influence

some_influence_edlevel = some_influence_df['EdLevel'].value_counts()
some_influence_edlevel

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          91
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       51
Some college/university study without earning a degree                                16
Associate degree (A.A., A.S., etc.)                                                    5
Professional degree (JD, MD, Ph.D, Ed.D, etc.)                                         3
Something else                                                                         2
Primary/elementary school                                                              2
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     2
Name: count, dtype: int64